In [ ]:
import polars as pl

In [ ]:
#Gold Glove Award Dataset
# Reading the Fielding.csv file
fielding = pl.read_csv("./data/lahman_1871-2023_csv/Fielding.csv")
fielding.head()

In [5]:
#Clean the Fielding Data
# Cleaning the Fielding data to include only relevant columns and data from 2013 onwards
fielding_clean = (fielding
                 .select(['playerID', 'yearID', 'lgID', 'POS'])
                 .filter(pl.col('yearID') >= 2013)
                 )
fielding_clean


playerID,yearID,lgID,POS
str,i64,str,str
"""aardsda01""",2013,"""NL""","""P"""
"""aardsda01""",2015,"""NL""","""P"""
"""abadfe01""",2013,"""NL""","""P"""
"""abadfe01""",2014,"""AL""","""P"""
"""abadfe01""",2015,"""AL""","""P"""
…,…,…,…
"""zuninmi01""",2022,"""AL""","""C"""
"""zuninmi01""",2023,"""AL""","""C"""
"""zychto01""",2015,"""AL""","""P"""


In [6]:
#Read in the Awards Data
# Reading the AwardsPlayers.csv file
awards_players = pl.read_csv("./data/lahman_1871-2023_csv/AwardsPlayers.csv")
awards_players.head()


playerID,awardID,yearID,lgID,tie,notes
str,str,i64,str,str,str
"""poseybu01""","""Lou Gehrig Memorial Award""",2019,"""ML""",null,null
"""klubeco01""","""Lou Gehrig Memorial Award""",2018,"""ML""",null,null
"""vottojo01""","""Lou Gehrig Memorial Award""",2017,"""ML""",null,null
"""altuvjo01""","""Lou Gehrig Memorial Award""",2016,"""ML""",null,null
"""grandcu01""","""Lou Gehrig Memorial Award""",2015,"""ML""",null,null


In [7]:
#Filter Awards Data for Gold Glove Awards
# Checking the name of the award and cleaning the Awards data to include only Gold Glove awards from 2013 onwards
awards_clean = (awards_players
                .select(['playerID', 'yearID', 'lgID', 'awardID', 'notes'])
                .filter(pl.col('yearID') >= 2013)
                .filter(pl.col('awardID') == 'Gold Glove')
         )
awards_clean


playerID,yearID,lgID,awardID,notes
str,i64,str,str,str
"""kineris01""",2020,"""AL""","""Gold Glove""","""3B"""
"""crawfjp01""",2020,"""AL""","""Gold Glove""","""SS"""
"""roberlu01""",2020,"""AL""","""Gold Glove""","""LF"""
"""gordoal01""",2020,"""AL""","""Gold Glove""","""CF"""
"""gallojo01""",2020,"""AL""","""Gold Glove""","""RF"""
…,…,…,…,…
"""barnhtu01""",2017,"""NL""","""Gold Glove""","""C"""
"""goldspa01""",2017,"""NL""","""Gold Glove""","""1B"""
"""lemahdj01""",2017,"""NL""","""Gold Glove""","""2B"""


In [8]:
#Join Fielding and Awards Data
# Left join of fielding and awards data to retain all players, adding labels for Gold Glove winners
gold_glove = (fielding_clean
              .join(awards_clean,
                    left_on = ['playerID', 'yearID', 'lgID', 'POS'],
                    right_on = ['playerID', 'yearID', 'lgID', 'notes'],
                    how = 'left'
                   )
              .with_columns(gold_glove = pl.when(pl.col('awardID') == 'Gold Glove')
                                           .then(pl.lit('Yes'))
                                           .otherwise(pl.lit('NO')),
                            Training_Validation = pl.when(pl.col('yearID') == 2023)
                                                    .then(pl.lit('Validation'))
                                                    .otherwise(pl.lit('Training'))
                                              )
             )
gold_glove


playerID,yearID,lgID,POS,awardID,gold_glove,Training_Validation
str,i64,str,str,str,str,str
"""aardsda01""",2013,"""NL""","""P""",null,"""NO""","""Training"""
"""aardsda01""",2015,"""NL""","""P""",null,"""NO""","""Training"""
"""abadfe01""",2013,"""NL""","""P""",null,"""NO""","""Training"""
"""abadfe01""",2014,"""AL""","""P""",null,"""NO""","""Training"""
"""abadfe01""",2015,"""AL""","""P""",null,"""NO""","""Training"""
…,…,…,…,…,…,…
"""zuninmi01""",2022,"""AL""","""C""",null,"""NO""","""Training"""
"""zuninmi01""",2023,"""AL""","""C""",null,"""NO""","""Validation"""
"""zychto01""",2015,"""AL""","""P""",null,"""NO""","""Training"""


In [9]:
#Save the Gold Glove Dataset
# Saving the gold glove dataset to a CSV file
gold_glove.write_csv("./data/gold_glove.csv")


In [10]:
# Salary Prediction Dataset
# Reading the Salaries.csv file and cleaning data from 2006 onwards
salaries = pl.read_csv("./data/lahman_1871-2023_csv/Salaries.csv")
salaries_clean = (salaries
                  .filter(pl.col('yearID') >= 2006)
                  .group_by(['playerID','yearID'])
                  .mean()
                  .drop(['teamID','lgID'])
                 )
salaries_clean.head()


playerID,yearID,salary
str,i64,f64
"""stewach01""",2012,482500.0
"""villaca01""",2012,2.2775e6
"""beimejo01""",2007,912500.0
"""dukesel01""",2007,380000.0
"""ruizca01""",2009,475000.0


In [11]:
# Read in and Clean Batting Data
# Reading and cleaning the Batting.csv file for data between 2006-2016
batting = pl.read_csv("./data/lahman_1871-2023_csv/Batting.csv")
batting_clean = (batting
                 .filter(pl.col('yearID') >= 2006,
                         pl.col('yearID') <= 2016)
                 .group_by(['playerID','yearID'])
                 .sum()
                 .drop(['teamID','lgID','G_old'])
                )
batting_clean


playerID,yearID,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""betanra01""",2012,1,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"""vizquom01""",2009,1,62,0,177,17,47,7,2,1,14,4,0,13,27,0,0,5,0,0
"""flahery01""",2014,1,102,0,281,33,62,15,1,7,32,1,0,22,68,2,5,3,1,3
"""lobatjo01""",2012,1,69,0,167,16,37,10,0,2,20,0,1,24,46,1,2,2,2,6
"""leecl02""",2011,1,35,0,75,6,15,2,0,2,7,1,0,1,26,0,0,5,1,2
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""rodrise01""",2008,1,59,0,167,18,34,8,1,3,10,3,1,14,55,0,3,2,1,3
"""elmorja01""",2015,1,51,0,141,10,29,5,0,2,16,1,1,12,25,1,0,2,3,6
"""howeljp01""",2013,1,67,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [12]:
#Read in and Clean Fielding Data
# Cleaning the Fielding.csv file for data between 2006-2016
fielding_agg = (fielding
                .filter(pl.col('yearID') >= 2006,
                         pl.col('yearID') <= 2016)
                 .group_by(['playerID','yearID'])
                 .sum()
                .drop(['teamID','lgID','POS','PB','WP','SB','CS','ZR'])
               )
fielding_agg


playerID,yearID,stint,G,GS,InnOuts,PO,A,E,DP
str,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""quentca01""",2013,1,69,69,1668,105,1,1,0
"""dukeza01""",2007,1,20,19,322,2,22,0,3
"""duffyma02""",2016,1,1,0,6,0,1,0,0
"""floripe01""",2015,2,20,4,205,12,21,2,3
"""mathido01""",2008,1,8,4,67,2,2,0,0
…,…,…,…,…,…,…,…,…,…
"""ryanky01""",2016,1,56,0,167,3,4,1,0
"""uptonbj01""",2013,1,118,111,2926,230,3,4,3
"""vargacl01""",2006,1,31,30,503,10,21,2,2


In [13]:
#Read in and Clean Pitching Data
# Reading and cleaning the Pitching.csv file for data between 2006-2016
pitching = pl.read_csv("./data/lahman_1871-2023_csv/Pitching.csv")
pitching_clean = (pitching
                  .filter(pl.col('yearID') >= 2006,
                          pl.col('yearID') <= 2016)
                  .group_by(['playerID','yearID'])
                  .sum()
                  .drop(['teamID','lgID'])
                 )
pitching_clean


playerID,yearID,stint,W,L,G,GS,CG,SHO,SV,IPouts,H,ER,HR,BB,SO,BAOpp,ERA,IBB,WP,HBP,BK,BFP,GF,R,SH,SF,GIDP
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""jenkich01""",2014,1,1,1,21,0,0,0,0,95,34,9,2,6,18,0.264,2.56,1,0,1,0,136,12,10,0,0,2
"""pattejo02""",2007,1,1,5,7,7,0,0,0,94,39,26,5,22,15,0.31,7.47,1,4,0,0,152,0,26,3,1,2
"""seaybo01""",2007,1,3,0,58,0,0,0,1,139,38,12,1,15,38,0.228,2.33,4,1,2,1,189,19,12,2,2,3
"""handbr01""",2011,1,1,8,12,12,0,0,0,180,53,28,10,35,38,0.241,4.2,1,0,1,1,263,0,32,4,3,2
"""sharpjo01""",2007,1,0,1,6,0,0,0,0,13,7,6,3,1,1,0.368,12.46,0,0,0,0,21,2,6,1,0,0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""morgaad01""",2016,1,2,11,23,21,0,0,0,340,141,76,23,29,95,0.303,6.04,3,2,4,0,507,1,81,3,4,8
"""marshja01""",2009,1,0,2,10,0,0,0,0,22,13,12,1,0,1,0.406,14.73,0,0,1,0,35,2,12,2,0,1
"""latosma01""",2014,1,5,5,16,16,0,0,0,307,92,37,9,26,74,0.24,3.25,2,1,2,0,420,0,42,8,1,6


In [14]:
# Join Batting, Fielding, and Pitching Data
batting_fielding_pitching = (batting_clean
                             .join(fielding_agg,
                                   on=['playerID', 'yearID'],
                                   how='full',
                                   suffix='_fielding'  # Add suffix for fielding columns to avoid duplicates
                                  )
                             .join(pitching_clean,
                                   on=['playerID', 'yearID'],
                                   how='full',
                                   suffix='_pitching'  # Add suffix for pitching columns to avoid duplicates
                                  )
                            )

batting_fielding_pitching


playerID,yearID,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,playerID_fielding,yearID_fielding,stint_fielding,G_fielding,GS,InnOuts,PO,A,E,DP,playerID_pitching,yearID_pitching,stint_pitching,W,L,G_pitching,GS_pitching,CG,SHO,SV,IPouts,H_pitching,ER,HR_pitching,BB_pitching,SO_pitching,BAOpp,ERA,IBB_pitching,WP,HBP_pitching,BK,BFP,GF,R_pitching,SH_pitching,SF_pitching,GIDP_pitching
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64
"""betanra01""",2012,1,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""betanra01""",2012,1,60,0,173,3,1,0,0,"""betanra01""",2012,1,1,4,60,0,0,0,31,173,53,18,6,12,57,0.241,2.81,4,0,0,1,236,53,19,2,2,3
"""vizquom01""",2009,1,62,0,177,17,47,7,2,1,14,4,0,13,27,0,0,5,0,0,"""vizquom01""",2009,3,63,45,1271,60,147,0,36,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""flahery01""",2014,1,102,0,281,33,62,15,1,7,32,1,0,22,68,2,5,3,1,3,"""flahery01""",2014,5,107,78,2208,104,202,10,35,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""lobatjo01""",2012,1,69,0,167,16,37,10,0,2,20,0,1,24,46,1,2,2,2,6,"""lobatjo01""",2012,1,66,50,1401,448,28,4,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""leecl02""",2011,1,35,0,75,6,15,2,0,2,7,1,0,1,26,0,0,5,1,2,"""leecl02""",2011,1,32,32,698,7,26,1,1,"""leecl02""",2011,1,17,8,32,32,6,6,0,698,197,62,18,42,238,0.229,2.4,0,0,6,0,920,0,66,6,4,20
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""rodrise01""",2008,1,59,0,167,18,34,8,1,3,10,3,1,14,55,0,3,2,1,3,"""rodrise01""",2008,3,56,47,1334,102,139,2,38,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""elmorja01""",2015,1,51,0,141,10,29,5,0,2,16,1,1,12,25,1,0,2,3,6,"""elmorja01""",2015,6,55,40,1107,202,36,5,20,"""elmorja01""",2015,1,0,0,1,0,0,0,0,3,3,1,1,0,0,0.5,9.0,0,0,0,0,6,0,1,0,0,0
"""howeljp01""",2013,1,67,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"""howeljp01""",2013,1,67,0,186,6,10,0,1,"""howeljp01""",2013,1,4,1,67,0,0,0,0,186,42,14,2,23,54,0.193,2.03,3,3,1,0,246,6,14,1,3,8


In [15]:
 #Merge Salaries with Performance Data
# Joining salaries with batting, fielding, and pitching performance data
salaries_df = (salaries_clean
               .join(batting_fielding_pitching,
                     on=['playerID', 'yearID'],
                     how='left')
               .with_columns(Training_Validation = pl.when(pl.col('yearID') == 2016)
                                                     .then(pl.lit('Validation'))
                                                     .otherwise(pl.lit('Training')))
              )
salaries_df

playerID,yearID,salary,stint,G,G_batting,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,playerID_fielding,yearID_fielding,stint_fielding,G_fielding,GS,InnOuts,PO,A,E,DP,playerID_pitching,yearID_pitching,stint_pitching,W,L,G_pitching,GS_pitching,CG,SHO,SV,IPouts,H_pitching,ER,HR_pitching,BB_pitching,SO_pitching,BAOpp,ERA,IBB_pitching,WP,HBP_pitching,BK,BFP,GF,R_pitching,SH_pitching,SF_pitching,GIDP_pitching,Training_Validation
str,i64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""stewach01""",2012,482500.0,1,55,0,141,15,34,8,0,1,13,2,0,10,21,0,1,3,2,1,"""stewach01""",2012,1,54,46,1186,379,25,4,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""villaca01""",2012,2.2775e6,1,38,0,2,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"""villaca01""",2012,1,38,16,376,7,11,0,0,"""villaca01""",2012,1,7,7,38,16,0,0,0,376,113,58,23,46,122,0.242,4.16,4,6,3,1,521,9,59,2,4,12,"""Training"""
"""beimejo01""",2007,912500.0,1,83,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,"""beimejo01""",2007,1,83,0,202,11,22,1,6,"""beimejo01""",2007,1,4,2,83,0,0,0,1,202,63,29,1,24,39,0.253,3.88,6,3,1,2,281,10,30,5,2,11,"""Training"""
"""dukesel01""",2007,380000.0,1,52,0,184,27,35,3,2,10,21,2,4,33,44,0,2,0,1,6,"""dukesel01""",2007,1,40,37,1037,87,3,0,1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""ruizca01""",2009,475000.0,1,107,0,322,32,82,26,1,9,43,3,2,47,39,8,4,4,2,8,"""ruizca01""",2009,1,107,100,2647,707,49,3,7,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""varitja01""",2006,1e7,1,103,0,365,46,87,19,2,12,55,1,2,46,87,7,2,1,2,10,"""varitja01""",2006,1,99,94,2467,647,28,4,3,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,"""Training"""
"""guevaca01""",2008,390000.0,1,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""guevaca01""",2008,1,10,0,37,0,2,0,0,"""guevaca01""",2008,1,1,0,10,0,0,0,0,37,13,8,2,9,11,0.265,5.84,2,3,0,0,60,6,9,1,1,1,"""Training"""
"""cedenxa01""",2014,507800.0,1,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,"""cedenxa01""",2014,1,9,0,21,0,1,0,0,"""cedenxa01""",2014,1,0,0,9,0,0,0,0,21,10,3,1,0,5,0.333,3.86,0,0,0,0,30,4,4,0,0,0,"""Training"""


In [16]:
#Save the Salary Dataset
# Save the salaries dataset to a CSV file
salaries_df.write_csv("./data/salaries_df.csv")


In [ ]:

# Import necessary libraries
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Load the gold glove dataset
gold_glove = pd.read_csv('./data/gold_glove.csv')  # Update the path to your data file


In [ ]:

}
